# ODP Bootstrap (fixed scale parameter) in Python # 
#### by tpedro ####

Import the required libraries.

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
from IPython.display import display
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt

Read in the triangle as a column from a CSV file; convert the OP and DP to be 0-indexed (Python convention).

In [2]:
tridata=pd.read_csv('C:/Users/tpedro/Documents/Python/paper1 example1.csv')
tridata=pd.DataFrame(tridata)

tridata['i']=tridata['OP']-min(tridata['OP'])
tridata['j']=tridata['DP']-1

Convert the columns to a triangle/matrix and then to a numpy array.

In [3]:
C_ij = pd.pivot_table(data=tridata, values='VALUE', index=['i'],columns=['j'], aggfunc=np.sum)
C_ij = C_ij.values # convert to a np array
C_ij = np.nan_to_num(C_ij) # convert nan to zeros, not needed

Define the required arrays (this is force of habit from an Igloo and VBA coder, it is not strictly necessary in Python!). 

Naming conventions used:
- "_ij" arrays are defined for each OP and DP combination, "_i" fo each OP and "_j" for each DP. 
- "_proj" arrays are the projections to ultimate.
- "_sim" indicates simulated items, these have an extra dimension. 
- "_inc" arrays are the incremental triangles.

In [4]:
n = C_ij.shape[0] # number of origin periods

latestj = np.arange(n,0,-1)-1 #number of developments in each origin period i.e. n, n-1, etc

sims = 50000 # number of simulations; MANUAL.

oneszeros = 1 * (C_ij > 0) # array of 1's where C_ij is greater than zero, assumes entries in the triangle are positive!

Out_OPs = [x+min(tridata['OP']) for x in range(n)] # OP labels
Out_DPs = [x+1 for x in range(n)] # DP labels

Here is our triangle.

In [5]:
pd.DataFrame(C_ij,index=Out_OPs,columns=Out_DPs)

,1,2,3,4,5,6,7,8,9,10
2005,5947.0,9668.0,10564.0,10772.0,10978.0,11041.0,11106.0,11121.0,11132.0,11148.0
2006,6347.0,9593.0,10316.0,10468.0,10536.0,10573.0,10625.0,10637.0,10648.0,0.0
2007,6269.0,9245.0,10092.0,10355.0,10508.0,10573.0,10627.0,10636.0,0.0,0.0
2008,5863.0,8546.0,9269.0,9459.0,9592.0,9681.0,9724.0,0.0,0.0,0.0
2009,5779.0,8524.0,9178.0,9451.0,9682.0,9787.0,0.0,0.0,0.0,0.0
2010,6185.0,9013.0,9586.0,9831.0,9936.0,0.0,0.0,0.0,0.0,0.0
2011,5600.0,8493.0,9057.0,9282.0,0.0,0.0,0.0,0.0,0.0,0.0
2012,5288.0,7728.0,8256.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013,5291.0,7649.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014,5676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Calculate observed development factors.

In [6]:
C_ij2 = np.roll(C_ij, -1, axis=1) # move entries one column to to the left
C_ij2[:,n-1] = 0 # set the final column of this array to zero (it is the first column of the original triangle)

f_ij = C_ij2 / (C_ij + 1 * (C_ij==0)) # the extra 1 * () term is to prevent division by zero

oneszeros_f = 1 * (C_ij2 > 0) # similar to other oneszeros array but for the f_ij

pd.DataFrame(f_ij,index=Out_OPs,columns=Out_DPs)

,1,2,3,4,5,6,7,8,9,10
2005,1.625694,1.092677,1.019690,1.019124,1.005739,1.005887,1.001351,1.000989,1.001437,0.0
2006,1.511423,1.075367,1.014734,1.006496,1.003512,1.004918,1.001129,1.001034,0.000000,0.0
2007,1.474717,1.091617,1.026060,1.014775,1.006186,1.005107,1.000847,0.000000,0.000000,0.0
2008,1.457616,1.084601,1.020498,1.014061,1.009279,1.004442,0.000000,0.000000,0.000000,0.0
2009,1.474996,1.076725,1.029745,1.024442,1.010845,0.000000,0.000000,0.000000,0.000000,0.0
2010,1.457235,1.063575,1.025558,1.010681,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2011,1.516607,1.066408,1.024843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2012,1.461422,1.068323,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2013,1.445662,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


Calculate chain ladder factors.

In [7]:
tempnum_j = np.nansum(C_ij2, axis=0) # sum the columns of the shifted triangle
tempden_j = np.nansum(C_ij * oneszeros_f, axis=0) # sum the columns of the original triangle. 
# The oneszeros_f eliminates the last entry of each column

tempnum_j [ tempden_j == 0 ] = 1 # replace with 1 where denominator is zero
tempden_j [ tempden_j == 0 ] = 1

f_j = tempnum_j/tempden_j # chain ladder factors

pd.DataFrame(f_j,index=Out_DPs,columns=['LDFs']).transpose()

,1,2,3,4,5,6,7,8,9,10
LDFs,1.492496,1.077786,1.022862,1.01485,1.006999,1.005111,1.001113,1.001011,1.001437,1.0


Calculate the age to ultimate factors.

In [8]:
tempf_j = np.flip(f_j, axis = 0) # reverse the order of the chain ladder factors

tempu_j = np.cumprod(tempf_j, axis = 0) # age to ultimate factors

u_j = np.flip(tempu_j, axis = 0) # reverse the order 

pd.DataFrame(u_j,index=Out_DPs,columns=['AtoU']).transpose()

,1,2,3,4,5,6,7,8,9,10
AtoU,1.696105,1.136422,1.054405,1.030838,1.015754,1.008695,1.003565,1.00245,1.001437,1.0


Calculate ultimates and reserves.

In [9]:
temp_diag = np.diagonal(np.fliplr(C_ij),0,0,1) # this is the latest diagonal from the triangle.

# note: the diagonal function as a default goes from top left to bottom right. fliplr is to go from bottom left to top right. 
# note: elements will be in wrong order with last OP first. This is why I am using the tempu_j from before.

Ultimate_i = temp_diag * tempu_j
R_i = temp_diag * (tempu_j - 1)
R=np.sum(R_i)
Ultimate = np.sum(Ultimate_i)

Calculate (unscaled) Pearson residuals.

In [10]:
C_ij_proj = np.einsum('i,j->ij',Ultimate_i,1/u_j) * oneszeros # this is the triangle implied by the LDFs

# note: the einsum (Einstein summation notation)is a handy function. The calculation is C_ij_proj = Ultimate_i / u_j. 
# note: The latest diagonal will be the same as C_ij.
# note: the * oneszeros is so that only the upper part of the triangle is kept


##### convert to incremental triangles #####
C_ij2 = np.roll(C_ij,1,axis=1) # this is the triangle shifted to the right. Using the same name from earlier.
C_ij2[:,0] = 0 # set the first column to zero as it will have the entries from the last column of the original triangle.

# note: C_ij2 = C_ij+1, with C_i1 = 0 for all i. 

C_ij_inc = (C_ij - C_ij2) * oneszeros # incremental triangle. The oneszeros is to get rid of the latest diagonal of C_ij 

C_ij_proj2 = np.roll(C_ij_proj,1,axis=1) # same for the projected triangle
C_ij_proj2[:,0]=0

C_ij_proj_inc = (C_ij_proj - C_ij_proj2) * oneszeros # same for the projected triangle


##### calculate residuals
r_ij = (C_ij_inc - C_ij_proj_inc) / (C_ij_proj_inc ** 0.5 + 1 * (C_ij_proj_inc == 0)) # the extra 1 * () term is to prevent division by zero
r_ij2 = r_ij * r_ij # square residuals

pd.DataFrame(r_ij,index=Out_OPs,columns=Out_DPs)

,1,2,3,4,5,6,7,8,9,10
2005,-7.717875e+00,8.506418,4.812710,-2.168256,3.582728,-1.575769,1.132306,0.751336,-0.072897,0.0
2006,7.575273e-01,2.690519,-0.254657,-5.208767,-6.907676,-4.254877,-0.276647,0.051960,0.074536,0.0
2007,-2.169375e-01,-2.155346,4.338581,2.093226,-0.048149,-0.987294,-0.003707,-0.820230,0.000000,0.0
2008,1.442615e+00,-2.829329,2.129647,-1.484018,-0.639515,2.653984,-0.917167,0.000000,0.000000,0.0
2009,-5.433004e-01,-2.270177,-0.835683,4.029590,7.444862,4.483944,0.000000,0.000000,0.000000,0.0
2010,3.041049e+00,-1.894424,-4.482384,1.769422,-3.349863,0.000000,0.000000,0.000000,0.000000,0.0
2011,-5.498866e-01,2.175767,-3.552943,1.217926,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2012,2.171320e+00,-1.744482,-2.779578,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2013,2.319167e+00,-3.304692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2014,-1.207199e-14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


Calculate scaling factor and apply to the residuals.

In [11]:
pts = n*(n+1)/2 # no. observations (points in the triangle)
ops = n-1 # no. origin periods for residuals
params = n-1 # no. parameters (i.e. LDFs; if curve is fitted then this will be the number of parameters for the curve)
dofs = pts - (ops + params) # no. degrees of freedom
pscale = np.nansum(r_ij2) / dofs # Pearson scale parameter
scale =  ( pts / (dofs * pscale) ) ** 0.5 # scalar to apply to residuals

r_ij_ = scale * r_ij # Pearson residuals (scaled)

Put the residuals into a 1-dimensional vector.

In [12]:
obs_i=np.copy(latestj) + 1 # observations in each origin period (incremental triangle entries)
obs_i[0]=latestj[0] # one less observation in first OP
obs_i[n-1]=0 # no observation in last OP
obs=int(np.nansum(obs_i)) # number of residuals
r_ij_lst=np.zeros(obs) # residuals as a list

counter=np.zeros(n) # cumulative entry number

for x in range(n-1): # no residual in final row of triangle
    minj=int(obs_i[x])    
    counter[x+1]=counter[x]+obs_i[x]
    for y in range(minj):
        r_ij_lst[y+int(counter[x])]= r_ij_[x,y] # could use append instead of loop?
        
# note: is there a smart way to do this without looping?

Random sampling of residuals to create pseudo incremental triangles. 

In [13]:
r_ij_sim = np.array([np.random.choice(r_ij_lst, size=C_ij.shape, replace=True) for item in range(sims)])

C_ij_inc_sim = r_ij_sim * ((pscale * C_ij_proj_inc) ** 0.5) + C_ij_proj_inc

Calculate CL factors for each pseudo triangle.

In [14]:
C_ij_sim = np.cumsum(C_ij_inc_sim,axis = 2) * oneszeros # convert to cumulative triangle. oneszeros is to so it doesnt go beyond the diagonal. 
C_ij_sim2 = np.cumsum(C_ij_inc_sim,axis = 2) * oneszeros_f # same as above but oneszeros_f means the last diagonal is removed.

tempnum_j = np.nansum(C_ij_sim, axis=1) # sum columns
tempden_j = np.nansum(C_ij_sim2, axis=1)
tempnum_j = np.roll(tempnum_j, -1, axis=1) # move items one column. 

tempnum_j [ tempden_j == 0 ] = 1 # replace with 1 where denominator is zero. Alternative to the 1 * () terms used above
tempden_j [ tempden_j == 0 ] = 1

f_j_sim = tempnum_j/tempden_j

##### Calculate the age to ultimate factors also #####
tempf_j = np.flip(f_j_sim, axis = 1) # reverse order for deriving age to ultimate factors with cumulative products, there probably is a single step way....

tempu_j = np.cumprod(tempf_j, axis = 1) 
u_j_sim = np.flip(tempu_j, axis = 1)

Calculate ultimate projections of pseudo triangles.

In [15]:
temp_a = 1 / u_j_sim # X_j = a - b
temp_b = np.roll(temp_a, 1, axis = 1)
temp_b[:,0] = 0
X_j_sim = temp_a - temp_b # NOT SURE WHAT THESE ARE CALLED!!!

temp_diag = np.diagonal(np.fliplr(C_ij_sim),0,1,2) # fliplr is to go from bottom left to top right. Elements will be in wrong order with last OP first.

temp_ult = temp_diag * u_j_sim # keeping diagonal in reverse order to make the maths easier 
Ultimate_i_proj_sim = np.flip(temp_ult, axis = 1) # reserve the order

temp = np.einsum('ij,ik->ijk',Ultimate_i_proj_sim,X_j_sim) # future incremental entries

# note: using an einsum again - workaround to do an outer product of the second dimension across all sims; takes two (sims,n) vectors and creates one (sims,n,n) matrix

oneszeros_temp = 1 - oneszeros # lower half of the triangle

temp = temp * oneszeros_temp # future incremental entries only

C_ij_inc_proj_sim = temp + r_ij_sim * (abs(pscale * temp) ** 0.5)

Calculate simulated reserves.

In [16]:
R_i_proj_sim = np.sum(C_ij_inc_proj_sim, axis = 2) # the reserves are the sum of the future incremental entries
R_proj_sim = np.sum(R_i_proj_sim, axis = 1)

Calculate overall RMSEP, process error and back out parameter error.

In [17]:
RMSEP_i=np.std(R_i_proj_sim,axis =0)
R_i_Mean=np.mean(R_i_proj_sim, axis=0)
RMSEP=np.std(R_proj_sim)
R_Mean=np.mean(R_proj_sim)

Proc_Error_i = (pscale * R_i)
Proc_Error = pscale * R

Param_Error_i = (RMSEP_i ** 2) - Proc_Error_i
Param_Error = (RMSEP ** 2) - Proc_Error

Set up and print outputs.

In [18]:
# Set up values for output: % Param/Proc error, CoVs etc
Proc_Error_Perc_i = Proc_Error_i / (RMSEP_i ** 2 + 1 * (RMSEP_i == 0)) * 100 #multiply by 100 as percentage
Param_Error_Perc_i = 100 - Proc_Error_Perc_i 
Proc_Error_Perc = Proc_Error / (RMSEP ** 2) * 100 #multiply by 100 as percentage
Param_Error_Perc = 100 - Proc_Error_Perc


Ultimate_CoV_i = RMSEP_i / Ultimate_i * 100 #multiply by 100 as percentage
Res_CoV_i = RMSEP_i / (R_i + 1 * (R_i==0)) * 100 #multiply by 100 as percentage
Ult_CoV = RMSEP / Ultimate * 100 #multiply by 100 as percentage
Res_CoV = RMSEP / R * 100 #multiply by 100 as percentage


# Set up output array
Out_Label = ['Ultimate', 'Reserve', 'RMSEP', 'Ult CoV', 'Res CoV', '% Param Err', '% Proc Err']
Out_OPs = [x+min(tridata['OP']) for x in range(n)]
Out_OPs_Tot = ['Total']
Out_OPs = Out_OPs + Out_OPs_Tot # add row of totals at the bottom

Out_Figures = [[Ultimate_i[x], R_i[x], RMSEP_i[x], Ultimate_CoV_i[x], Res_CoV_i[x], Param_Error_Perc_i[x], Proc_Error_Perc_i[x]] for x in range(n)]
Out_Figures_Tot = [[Ultimate,R,RMSEP,Ult_CoV,Res_CoV, Param_Error_Perc, Proc_Error_Perc]]

Out_Figures = Out_Figures + Out_Figures_Tot # add row of totals at the bottom

pd.DataFrame(Out_Figures,index=Out_OPs,columns=Out_Label).round(1) # round to 1 DP



,Ultimate,Reserve,RMSEP,Ult CoV,Res CoV,% Param Err,% Proc Err
2005,11148.0,0.0,0.0,0.0,0.0,100.0,0.0
2006,10663.3,15.3,21.9,0.2,143.0,54.4,45.6
2007,10662.1,26.1,27.0,0.3,103.5,48.8,51.2
2008,9758.7,34.7,29.4,0.3,84.8,42.6,57.4
2009,9872.1,85.1,42.4,0.4,49.9,32.5,67.5
2010,10092.5,156.5,55.6,0.6,35.5,27.5,72.5
2011,9568.2,286.2,73.3,0.8,25.6,23.9,76.1
2012,8705.2,449.2,90.9,1.0,20.2,22.3,77.7
2013,8692.5,1043.5,141.3,1.6,13.5,25.3,74.7
2014,9627.1,3951.1,333.4,3.5,8.4,49.2,50.8
